In [1]:
import requests
import json
import arcgis
from arcgis.gis import GIS, Item
import os
from PIL import Image as PILImage
import tempfile
from io import BytesIO
from copy import deepcopy

In [2]:
# Print the version of the arcgis module
print(f"Running ArcGIS API for Python version: {arcgis.__version__}")
agoNotebook = False
# Define the GIS
if agoNotebook == False:
    try:
        import keyring
        service_name = "system" # Use the default local credential store
        success = False # Set initial state

        # Ask for the username
        while success == False:
            username_for_keyring = input("Enter your ArcGIS Online username:") # If you are using VS Code, the text input dialog box appears at the top of the window
            # Get the credential object
            credential = keyring.get_credential(service_name, username_for_keyring)
            # Check if the username is in the credential store
            if credential is None:
                print(f"'{username_for_keyring}' is not in the local system's credential store. Try another username.")
            # Retrieve the password, login and set the GIS portal
            else:
                password_from_keyring = keyring.get_password("system", username_for_keyring)
                portal_url = 'https://www.arcgis.com'  
                gis = GIS(portal_url, username=username_for_keyring, password=password_from_keyring)
                success = True
                # Print a success message with username and user's organization role
                print(f"Successfully logged in as: {gis.properties.user.username} (role: {gis.properties.user.role} userType: {gis.properties.user.userLicenseTypeId})")
    except ImportError:
        print("The 'keyring' module is not installed. Please install it using 'pip install keyring'.")
        print("Before re-running this cell, open a command line window on your machine and run the command:")
        print("# python -m keyring set system <your_ago_username>")
        print("If using Windows Powershell, use:")
        print("# ./python -m keyring set system <your_ago_username>")
        print("You will be prompted to enter your password")
        print("When you hit Enter/Return the password will be saved to your local credential store.")
else:
    gis = GIS("home")

Running ArcGIS API for Python version: 2.4.1.3
Successfully logged in as: dasbury_storymaps (role: org_admin userType: GISProfessionalAdvUT)


In [3]:


def remove_failed_service(webmap_json, failed_url):
    # Remove from operationalLayers
    if 'operationalLayers' in webmap_json:
        webmap_json['operationalLayers'] = [
            lyr for lyr in webmap_json['operationalLayers']
            if not lyr.get('url', '').startswith(failed_url)
        ]
    # Remove from baseMapLayers
    if 'baseMap' in webmap_json and 'baseMapLayers' in webmap_json['baseMap']:
        webmap_json['baseMap']['baseMapLayers'] = [
            lyr for lyr in webmap_json['baseMap']['baseMapLayers']
            if not lyr.get('url', '').startswith(failed_url)
        ]
    return webmap_json

# For downloads, use in-memory BytesIO where possible
def create_webmap_thumbnail(webmap_item, thumbnail_path, default_thumbnail_path, gis=None):
    try:
        url = "https://utility.arcgisonline.com/arcgis/rest/services/Utilities/PrintingTools/GPServer/Export%20Web%20Map%20Task/execute"
        # Get your web map JSON (from an Item)
        webmap_json = webmap_item.get_data()
        bookmarks = webmap_json.get('bookmarks', [])
        if bookmarks:
            for bm in bookmarks:
                bm_name = bm.get('name')
                bm_extent = bm.get('extent')
                if not bm_extent:
                    continue

            # Deep copy the web map JSON so each print uses a fresh copy
            webmap_json_bm = deepcopy(webmap_json)
            webmap_json_bm = remove_failed_service(webmap_json_bm, failed_service_url)
            # Set the extent for this bookmark
            if 'mapOptions' not in webmap_json_bm:
                webmap_json_bm['mapOptions'] = {}
            webmap_json_bm['mapOptions']['extent'] = bm_extent

            if 'exportOptions' not in webmap_json_bm:
                webmap_json_bm['exportOptions'] = {
                    "outputSize": [800, 600],  # width, height in pixels
                    "dpi": 96
                }

            params = {
                "f": "json",
                "Web_Map_as_JSON": json.dumps(webmap_json_bm),
                "Format": "PNG32",
                "Layout_Template": "MAP_ONLY"
            }

            response = requests.post(url, data=params)
            result = response.json()
            if 'results' in result:
                image_url = result['results'][0]['value']['url']
                #print(f"Bookmark '{bm_name}' exported image URL: {image_url}")
            if 'error' in result:
                failed_layer = result['error']['details'][0]
                failed_service_url = failed_layer.split(' at ')[-1]
                removed_layer_json = remove_failed_service(webmap_json_bm, failed_service_url)
                webmap_json_bm = removed_layer_json


            else:
                print(f"Error exporting bookmark '{bm_name}': {result}")
            return image_url
    except Exception:
        print("Thumbnail download failed; using default.")
        return default_thumbnail_path

In [6]:
default_thumbnail_path = "https://cdn-a.arcgis.com/cdn/1BE082D/js/arcgis-app-components/arcgis-app/assets/arcgis-item-thumbnail/storymap.png"
item_id = "0bb11c0469f042b3afaf8b0d76572822"
webmap_item = gis.content.get(item_id)
thumbnail_paths = []
image_url = create_webmap_thumbnail(webmap_item, default_thumbnail_path, default_thumbnail_path, gis)
thumbnail_paths.append(image_url)
response = requests.get(image_url)
if response.status_code == 200:
    with open("thumbnail.png", "wb") as f:
        f.write(response.content)
    print("Image saved as thumbnail.png")
else:
    print(f"Failed to download image: {response.status_code}")

Thumbnail download failed; using default.
Image saved as thumbnail.png
